# Оцениваем регрессии на данных HS0 в спецификациях:
- Сырые данные ~ ковид и ограничения + контроль на дамми
- Сырые данные ~ ковид и ограничения + контроль на дамми | инструментальные переменные

Для второй спецификации считаем бутстраповские доверительные интервалы

In [1]:
library(arrow)
library(stargazer)
library(tidyverse)
library(plotly)
library(fixest)
signif.code = signif.code = c("***" = 0.001, "**" = 0.01, "*" = 0.05, "." = 0.10)


Присоединяю пакет: 'arrow'


Следующий объект скрыт от 'package:utils':

    timestamp



Please cite as: 


 Hlavac, Marek (2022). stargazer: Well-Formatted Regression and Summary Statistics Tables.

 R package version 5.2.3. https://CRAN.R-project.org/package=stargazer 


Warning message:
"пакет 'tidyverse' был собран под R версии 4.4.2"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::duration() masks arrow::duration()
✖ dplyr::filter()       masks stats::filter()
✖ dplyr::lag()          masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
"пакет 'plotly' был собран под R версии 4.4.2"

Присоеди

## Сырые данные

Под каждой регрессией вывожу summary с кластеризацией ошибок. В конце блока сравниваю робастность результата по используемым ошибкам.

In [3]:
# Скачиваю данные и переназываю переменные
df = read_parquet('../data/processed_data/trade_pvi_hs0_x_covid.parquet.gzip')
# Переменная pair нужна, чтобы считать НАС стандартные ошибки
df = df %>% mutate(pair = paste(i, j))

df$deaths_i = df$deaths
df$deaths_j = df$`deaths:r_j`

df$median_age_i = df$median_age
df$median_age_j = df$`median_age:r_j`

df$average_surface_temperature_i = df$average_surface_temperature
df$average_surface_temperature_j = df$`average_surface_temperature:r_j`

df$C1M_School_closing_i = df$C1M_School_closing
df$C1M_School_closing_j = df$`C1M_School_closing:r_j`

df$population_density_i = df$population_density
df$population_density_j = df$`population_density:r_j`

df$StringencyIndex_Average_i = df$StringencyIndex_Average
df$StringencyIndex_Average_j = df$`StringencyIndex_Average:r_j`

df$share_1_members_i = df$share_1_members
df$share_1_members_j = df$`share_1_members:r_j`

df$H6M_Facial_Coverings_i = df$H6M_Facial_Coverings
df$H6M_Facial_Coverings_j = df$`H6M_Facial_Coverings:r_j`

df$C8EV_International_travel_controls_i = df$C8EV_International_travel_controls
df$C8EV_International_travel_controls_j = df$`C8EV_International_travel_controls:r_j`
df %>% head()

t,i,j,pvi,country,t:r_i,cases,deaths,tests,C1M_School_closing,⋯,population_density_i,population_density_j,StringencyIndex_Average_i,StringencyIndex_Average_j,share_1_members_i,share_1_members_j,H6M_Facial_Coverings_i,H6M_Facial_Coverings_j,C8EV_International_travel_controls_i,C8EV_International_travel_controls_j
<int>,<chr>,<chr>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2020,BGD,SVK,1.0461535,BGD,2020,8.303453,0.121436464,NA,2.38524580,⋯,1277.54493,113.53634,66.01172,45.50587,2.85,25.33,2.3606558,2.2459016,2.491803,2.128415
2020,BRA,TCA,0.7051130,BRA,2020,97.838812,2.502044199,NA,2.41803288,⋯,24.96499,46.72211,57.50563,NA,12.00,33.59,3.0601094,NA,1.879781,NA
2021,BDI,TZA,3.0319083,BDI,2021,5.386493,0.002493151,NA,0.07123288,⋯,504.88633,70.93070,16.09011,12.24427,6.04,9.63,1.2273972,0.7369863,1.383562,1.495890
2020,IRL,GHA,1.8881887,IRL,2020,46.045442,1.205662983,NA,1.81693995,⋯,72.36728,140.14771,56.54366,42.53210,22.02,21.48,0.9234973,2.3551912,1.502732,2.300546
2020,IND,JOR,0.6454931,IND,2020,19.743646,0.286104972,NA,2.18579245,⋯,471.75515,122.36444,64.76656,58.29060,NA,4.36,2.9371586,0.9508197,3.131148,2.877049
2021,THA,UKR,1.7877377,THA,2021,84.168932,0.821890411,NA,2.04657531,⋯,140.39682,76.45606,55.72677,55.13792,21.59,24.96,3.2465754,2.5999999,2.591781,1.586301


In [4]:
df$C3M_Cancel_public_events_i = df$C3M_Cancel_public_events
df$C3M_Cancel_public_events_j = df$`C3M_Cancel_public_events:r_j`

In [5]:
# FE (т.е. оценка на панельных данных с использованием внутригрупповой дисперсии)
fe_no_iv = feols(log(pvi) ~ log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j| i + j + year,
 data = df)
fe_no_iv %>% summary(cluster = ~ i+j)

NOTE: 13,955 observations removed because of NA and infinite values (LHS: 645, RHS: 13,568).



OLS estimation, Dep. Var.: log(pvi)
Observations: 56,352
Fixed-effects: i: 175,  j: 175,  year: 3
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|)
log(deaths_i)                        -0.003444   0.005978 -0.576110 0.565285
log(deaths_j)                        -0.008151   0.006454 -1.262939 0.208301
C8EV_International_travel_controls_i -0.019261   0.015293 -1.259528 0.209527
C8EV_International_travel_controls_j -0.031013   0.016185 -1.916170 0.056983
                                        
log(deaths_i)                           
log(deaths_j)                           
C8EV_International_travel_controls_i    
C8EV_International_travel_controls_j .  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.14948     Adj. R2: 0.066016
                Within R2: 1.691e-4

In [6]:
# Добавляю гетерогенность эффектов во времени
heterogenous_fe = feols(log(pvi) ~ year/log(deaths_i) + year/log(deaths_j) +
year/C8EV_International_travel_controls_i + year/C8EV_International_travel_controls_j|
i + j ,
data = df)
heterogenous_fe %>% summary(cluster = ~ i+j)

NOTE: 13,955 observations removed because of NA and infinite values (LHS: 645, RHS: 13,568).



OLS estimation, Dep. Var.: log(pvi)
Observations: 56,352
Fixed-effects: i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                               Estimate Std. Error   t value
year2021                                      -0.083139   0.107813 -0.771142
year2022                                      -0.039314   0.121528 -0.323500
year2020:log(deaths_i)                        -0.002528   0.006974 -0.362575
year2021:log(deaths_i)                        -0.005490   0.006618 -0.829478
year2022:log(deaths_i)                        -0.004427   0.008075 -0.548191
year2020:log(deaths_j)                        -0.017383   0.007751 -2.242750
year2021:log(deaths_j)                        -0.011549   0.009057 -1.275226
year2022:log(deaths_j)                        -0.000355   0.008111 -0.043791
year2020:C8EV_International_travel_controls_i -0.039234   0.029757 -1.318511
year2021:C8EV_International_travel_controls_i -0.005611   0.015257 -0.367740
year2022:C8EV_International_t

### Инструментальные переменные на панельных данных

In [7]:
# Спецификация с фиксированными эффектами, куда я добавил инструменты, чтобы убрать потенциальную двустороннюю связь. 

fe_iv = feols(log(pvi) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j|
 log(deaths_i) + log(deaths_j) ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df
 )
fe_iv %>% summary(cluster = ~ i+j)

NOTE: 14,291 observations removed because of NA and infinite values (LHS: 645, RHS: 9,802, IV: 6,067/10,661).



TSLS estimation - Dep. Var.: log(pvi)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(pvi)
Observations: 56,016
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|) 
fit_log(deaths_i)                    -0.016405   0.025204 -0.650906  0.51597 
fit_log(deaths_j)                    -0.009376   0.021768 -0.430749  0.66718 
C8EV_International_travel_controls_i -0.024592   0.016748 -1.468357  0.14381 
C8EV_International_travel_controls_j -0.028803   0.017509 -1.645023  0.10177 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.14939     Adj. R2: 0.065875
                Within R2: 7.592e-5
F-test (1st stage), log(deaths_i): stat = 945.9     , p < 2.2e-16 , on 6 and 55,659 DoF.
F-test (1s

### Точечная оценка руками

In [8]:
# Т.к. данные корректные, это одно и то же
df %>% distinct(t, i, deaths_i, C8EV_International_travel_controls_i)  %>% count() %>% print()
df %>% distinct(t, i)  %>% count() %>% print()

# A tibble: 1 × 1
      n
  <int>
1   660
# A tibble: 1 × 1
      n
  <int>
1   660


In [9]:
# Дистинкт он вручную
iv_df = df %>% group_by(year, i) %>% 
summarise(deaths_i = min(deaths_i),
          C8EV_International_travel_controls_i = min(C8EV_International_travel_controls_i),
          C1M_School_closing_i = min(C1M_School_closing_i),
          median_age_i = min(median_age_i),
          population_density_i = min(population_density_i),
          H6M_Facial_Coverings = min(H6M_Facial_Coverings),
          C3M_Cancel_public_events = min(C3M_Cancel_public_events),
          .groups = 'drop') 
head(iv_df)

year,i,deaths_i,C8EV_International_travel_controls_i,C1M_School_closing_i,median_age_i,population_density_i,H6M_Facial_Coverings,C3M_Cancel_public_events
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2020,ABW,1.20447514,2.051913,1.385246,40.399,603.26111,1.054645,1.4153005
2020,AFG,0.14687845,1.461749,1.748634,16.411,59.90062,1.743169,0.9125683
2020,AGO,0.03096685,3.248634,2.079235,16.321,26.83174,2.467213,1.3005464
2020,AIA,0.00000000,NA,NA,36.071,NA,NA,NA
2020,ALB,1.10790055,1.836066,1.653005,35.127,103.57113,1.991803,1.6284153
2020,AND,2.87596685,1.557377,1.229508,42.125,164.63830,2.081967,1.1202186


In [10]:
# Оцениваю регрессию первого шага
set.seed(123)
first_stage = feols(log(deaths_i) ~ C8EV_International_travel_controls_i + 
C1M_School_closing_i + 
median_age_i + 
population_density_i
  | year+i,
 data = iv_df)

first_stage %>% wald(.,keep = 'C1M_School_closing_i|median_age_i|population_density_i', vcov = ~i)  # Проверка релевантности инструментов
first_stage  %>% summary(cluster = ~ i)

NOTE: 156 observations removed because of NA and infinite values (LHS: 74, RHS: 130).



Wald test, H0: joint nullity of C1M_School_closing_i, median_age_i and population_density_i
 stat = 14.0, p-value = 1.385e-8, on 3 and 323 DoF, VCOV: Clustered (i).

OLS estimation, Dep. Var.: log(deaths_i)
Observations: 504
Fixed-effects: year: 3,  i: 175
Standard-errors: Clustered (i) 
                                      Estimate Std. Error  t value   Pr(>|t|)
C8EV_International_travel_controls_i -0.612210   0.126406 -4.84322 2.8097e-06
C1M_School_closing_i                  0.552634   0.142851  3.86862 1.5452e-04
median_age_i                          0.722052   0.223577  3.22955 1.4824e-03
population_density_i                 -0.002437   0.001035 -2.35422 1.9677e-02
                                        
C8EV_International_travel_controls_i ***
C1M_School_closing_i                 ***
median_age_i                         ** 
population_density_i                 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.793277     Adj. R2: 0.784621
                 Within R2: 0.123806

In [11]:
# Предикт регрессии первого шага
new_iv_df = iv_df[, c('i', 'year', 'deaths_i', 'C8EV_International_travel_controls_i', 'C1M_School_closing_i',
                      'median_age_i', 'population_density_i')]  %>% drop_na()  %>% filter(deaths_i > 0)  %>% select(year, i) # Т.к. логарифм нуля это бесконечность
new_iv_df$predicted_log_deaths_i = first_stage %>% predict()
new_iv_df %>% head()

year,i,predicted_log_deaths_i
<chr>,<chr>,<dbl>
2020,ABW,0.40567115
2020,AFG,-1.99526479
2020,AGO,-3.32115465
2020,ALB,-0.66682830
2020,AND,0.03465424
2020,ARE,-2.10135519


In [12]:
# Джоиню предикт на исходный датафрейм
df_w_predictions = df %>% 
merge(., new_iv_df, by = c('year', 'i')) %>% 
merge(., new_iv_df, by.x = c('year', 'j'), by.y = c('year', 'i')) 
names(df_w_predictions)[names(df_w_predictions) == 'predicted_log_deaths_i.x'] <- 'predicted_log_deaths_i'
names(df_w_predictions)[names(df_w_predictions) == 'predicted_log_deaths_i.y'] <- 'predicted_log_deaths_j'
df_w_predictions %>% head()

,year,j,i,t,pvi,country,t:r_i,cases,deaths,tests,⋯,share_1_members_i,share_1_members_j,H6M_Facial_Coverings_i,H6M_Facial_Coverings_j,C8EV_International_travel_controls_i,C8EV_International_travel_controls_j,C3M_Cancel_public_events_i,C3M_Cancel_public_events_j,predicted_log_deaths_i,predicted_log_deaths_j
,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2020,ABW,SWE,2020,0.4421886,SWE,2020,110.13782,2.455691,NA,⋯,NA,21.37,0.000000,1.054645,2.500000,2.051913,1.565574,1.4153,0.03743995,0.4056712
2,2020,ABW,AUT,2020,0.4519360,AUT,2020,105.05586,2.148066,NA,⋯,36.29,21.37,1.786885,1.054645,2.442623,2.051913,1.333333,1.4153,0.05819797,0.4056712
3,2020,ABW,CZE,2020,0.7562766,CZE,2020,174.71884,2.942072,NA,⋯,32.51,21.37,2.193989,1.054645,2.633880,2.051913,1.013661,1.4153,0.50476226,0.4056712
4,2020,ABW,CRI,2020,1.0718229,CRI,2020,89.18359,1.143204,NA,⋯,12.07,21.37,1.218579,1.054645,2.322404,2.051913,1.622951,1.4153,-0.33880730,0.4056712
5,2020,ABW,GBR,2020,0.7566244,GBR,2020,94.98970,3.665912,NA,⋯,30.58,21.37,1.579235,1.054645,1.153005,2.051913,1.573771,1.4153,1.30345677,0.4056712
6,2020,ABW,CHL,2020,0.4039197,CHL,2020,84.54039,2.317541,NA,⋯,20.17,21.37,1.464481,1.054645,2.934426,2.051913,1.590164,1.4153,0.55502878,0.4056712


In [13]:
# Регрессия второго шага
second_stage = feols(log(pvi) ~ 
 predicted_log_deaths_i + predicted_log_deaths_j +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j,
 data = df_w_predictions)
second_stage %>% summary(cluster = ~ i+j)

NOTE: 385 observations removed because of NA and infinite values (LHS: 385).



OLS estimation, Dep. Var.: log(pvi)
Observations: 56,016
Fixed-effects: year: 3,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|) 
predicted_log_deaths_i               -0.017456   0.026622 -0.655691  0.51289 
predicted_log_deaths_j               -0.009221   0.022896 -0.402738  0.68764 
C8EV_International_travel_controls_i -0.026431   0.017666 -1.496195  0.13641 
C8EV_International_travel_controls_j -0.029285   0.017943 -1.632098  0.10447 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.14936     Adj. R2: 0.065923
                Within R2: 1.275e-4

In [14]:
# Экспорт в латех
etable(first_stage, vcov = ~i, signif.code=signif.code, tex=TRUE)
etable(second_stage, vcov = ~i+j, signif.code=signif.code, tex=TRUE)

\begingroup
\centering
\begin{tabular}{lc}
   \tabularnewline \midrule \midrule
   Dependent Variable:                          & log(deaths\_i)\\   
   Model:                                       & (1)\\  
   \midrule
   \emph{Variables}\\
   C8EV\_International\_travel\_controls\_i     & -0.6122$^{***}$\\   
                                                & (0.1264)\\   
   C1M\_School\_closing\_i                      & 0.5526$^{***}$\\   
                                                & (0.1429)\\   
   median\_age\_i                               & 0.7221$^{**}$\\   
                                                & (0.2236)\\   
   population\_density\_i                       & -0.0024$^{*}$\\   
                                                & (0.0010)\\   
   \midrule
   \emph{Fixed-effects}\\
   year                                         & Yes\\  
   i                                            & Yes\\  
   \midrule
   \emph{Fit statistics}\\
   Observations               

\begingroup
\centering
\begin{tabular}{lc}
   \tabularnewline \midrule \midrule
   Dependent Variable:                          & log(pvi)\\  
   Model:                                       & (1)\\  
   \midrule
   \emph{Variables}\\
   predicted\_log\_deaths\_i                    & -0.0175\\   
                                                & (0.0266)\\   
   predicted\_log\_deaths\_j                    & -0.0092\\   
                                                & (0.0229)\\   
   C8EV\_International\_travel\_controls\_i     & -0.0264\\   
                                                & (0.0177)\\   
   C8EV\_International\_travel\_controls\_j     & -0.0293\\   
                                                & (0.0179)\\   
   \midrule
   \emph{Fixed-effects}\\
   year                                         & Yes\\  
   i                                            & Yes\\  
   j                                            & Yes\\  
   \midrule
   \emph{Fit statistics}\\
   Obs

### Разные наборы инструментов

In [15]:
# List of instrumental variables
instrumental_variables <- c( "C1M_School_closing", "median_age", "population_density")

# Generate all subsets
all_subsets <- lapply(0:length(instrumental_variables), function(n) combn(instrumental_variables, n, simplify = FALSE))

# Flatten the list of subsets
all_subsets <- unlist(all_subsets, recursive = FALSE)

# Transform subsets to strings with _i and _j suffixes
transformed_subsets <- sapply(all_subsets, function(subset) {
  if (length(subset) == 0) return("")  # Handle empty subset
  # Create pairs for each variable and collapse with " + "
  paste(
    unlist(lapply(subset, function(var) paste0(var, c("_i", "_j"))), 
         recursive = TRUE),
    collapse = " + "
  )
})

# Remove empty string for empty subset (optional)
transformed_subsets <- transformed_subsets[transformed_subsets != ""]

formulas = lapply(transformed_subsets, function(subset){
    formula(paste("log(pvi) ~  1 + C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j | log(deaths_i) + log(deaths_j) ~", subset))
})
formulas[3][[1]]

log(pvi) ~ 1 + C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | 
    year + i + j | log(deaths_i) + log(deaths_j) ~ population_density_i + 
    population_density_j
<environment: 0x000002029178d118>

In [16]:
headers <- sapply(all_subsets, function(subset) {
  if (length(subset) == 0) return("")  # Handle empty subset
  # Create pairs for each variable and collapse with " + "
  paste(
    unlist(subset, recursive = TRUE),
    collapse = " + "
  )
})
headers[2:length(headers)]

[1] "C1M_School_closing"                                  
[2] "median_age"                                          
[3] "population_density"                                  
[4] "C1M_School_closing + median_age"                     
[5] "C1M_School_closing + population_density"             
[6] "median_age + population_density"                     
[7] "C1M_School_closing + median_age + population_density"

In [17]:
replace_vars_with_numbers <- function(headers) {
  # Define all unique variables
  variables <- c("C1M_School_closing",
               "median_age",
               "population_density")
  
  # Create a named vector for mapping
  var_numbers <- setNames(as.character(seq_along(variables)), variables)
  
  # Split and replace components
  sapply(headers, function(h) {
    components <- unlist(strsplit(h, " \\+ "))
    numbers <- var_numbers[components]
    paste(numbers, collapse = " + ")
  })
}
numbered_headers <- replace_vars_with_numbers(headers[2:length(headers)])
numbered_headers

C1M_School_closing 
                                                 "1" 
                                          median_age 
                                                 "2" 
                                  population_density 
                                                 "3" 
                     C1M_School_closing + median_age 
                                             "1 + 2" 
             C1M_School_closing + population_density 
                                             "1 + 3" 
                     median_age + population_density 
                                             "2 + 3" 
C1M_School_closing + median_age + population_density 
                                         "1 + 2 + 3"

In [18]:
list_of_iv_models = list()
for (i in 1:length(formulas)){
    list_of_iv_models[[i]] = feols(formulas[i][[1]], data = df)
}
etable(list_of_iv_models, vcov=~i+j, signif.code=signif.code, fitstat=~ . + sargan.p + wh.p + ivf,
    headers=numbered_headers, tex=TRUE)
etable(list_of_iv_models, vcov=~i+j, signif.code=signif.code, fitstat=~ . + sargan.p + wh.p + ivf,
    headers=numbered_headers, tex=FALSE)

NOTE: 13,955 observations removed because of NA and infinite values (LHS: 645, RHS: 9,802, IV: 6,067/9,802).

NOTE: 13,955 observations removed because of NA and infinite values (LHS: 645, RHS: 9,802, IV: 6,067/47).

NOTE: 14,291 observations removed because of NA and infinite values (LHS: 645, RHS: 9,802, IV: 6,067/2,257).

NOTE: 13,955 observations removed because of NA and infinite values (LHS: 645, RHS: 9,802, IV: 6,067/9,802).

NOTE: 14,291 observations removed because of NA and infinite values (LHS: 645, RHS: 9,802, IV: 6,067/10,661).

NOTE: 14,291 observations removed because of NA and infinite values (LHS: 645, RHS: 9,802, IV: 6,067/2,257).

NOTE: 14,291 observations removed because of NA and infinite values (LHS: 645, RHS: 9,802, IV: 6,067/10,661).



\begingroup
\centering
\begin{tabular}{lccccccc}
   \tabularnewline \midrule \midrule
   Dependent Variable: & \multicolumn{7}{c}{log(pvi)}\\
                                                & 1        & 2             & 3               & 1 + 2         & 1 + 3         & 2 + 3         & 1 + 2 + 3 \\   
   Model:                                       & (1)      & (2)           & (3)             & (4)           & (5)           & (6)           & (7)\\  
   \midrule
   \emph{Variables}\\
   log(deaths\_i)                               & -0.0230  & 0.0050        & -0.1518$^{***}$ & -0.0134       & -0.0282       & -0.0096       & -0.0164\\   
                                                & (0.0318) & (0.0255)      & (0.0345)        & (0.0245)      & (0.0314)      & (0.0307)      & (0.0252)\\   
   log(deaths\_j)                               & 0.0085   & -0.0320       & -0.0751         & -0.0068       & 0.0025        & -0.0367       & -0.0094\\   
                                             

,,model 1,model 2,model 3,model 4,model 5,model 6,model 7
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,,1,2,3,1 + 2,1 + 3,2 + 3,1 + 2 + 3
2,Dependent Var.:,log(pvi),log(pvi),log(pvi),log(pvi),log(pvi),log(pvi),log(pvi)
3,,,,,,,,
4,log(deaths_i),-0.0230 (0.0318),0.0050 (0.0255),-0.1518*** (0.0345),-0.0134 (0.0245),-0.0282 (0.0314),-0.0096 (0.0307),-0.0164 (0.0252)
5,log(deaths_j),0.0085 (0.0316),-0.0320 (0.0228),-0.0751 (0.0806),-0.0068 (0.0215),0.0025 (0.0315),-0.0367 (0.0236),-0.0094 (0.0218)
6,C8EV_International_travel_controls_i,-0.0276 (0.0172),-0.0153 (0.0188),-0.0785* (0.0316),-0.0233 (0.0162),-0.0297. (0.0176),-0.0212 (0.0208),-0.0246 (0.0168)
7,C8EV_International_travel_controls_j,-0.0234 (0.0192),-0.0416* (0.0194),-0.0557 (0.0411),-0.0303. (0.0171),-0.0234 (0.0194),-0.0409* (0.0203),-0.0288 (0.0175)
8,Fixed-Effects:,----------------,-----------------,-------------------,-----------------,-----------------,-----------------,----------------
9,year,Yes,Yes,Yes,Yes,Yes,Yes,Yes


### Робастность с выкидываем одного года

In [19]:
fe_iv_no2020 = feols(log(pvi) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j|
 log(deaths_i) + log(deaths_j) ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df %>% filter(year != "2020")
 )

fe_iv_no2020 %>% summary(cluster = ~ i+j)

feols(log(pvi) ~  1 + log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j,
 data = df %>% filter(year != "2020")) %>% summary(cluster = ~ i+j)

NOTE: 8,651 observations removed because of NA and infinite values (LHS: 448, RHS: 6,383, IV: 2,686/7,242).



TSLS estimation - Dep. Var.: log(pvi)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(pvi)
Observations: 37,734
Fixed-effects: year: 2,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|) 
fit_log(deaths_i)                    -0.018923   0.043527 -0.434742  0.66429 
fit_log(deaths_j)                    -0.010297   0.033263 -0.309580  0.75725 
C8EV_International_travel_controls_i -0.038285   0.025561 -1.497803  0.13600 
C8EV_International_travel_controls_j -0.018656   0.022651 -0.823649  0.41127 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.20388     Adj. R2: 0.064592
                Within R2: 2.457e-5
F-test (1st stage), log(deaths_i): stat = 526.6    , p < 2.2e-16, on 6 and 37,377 DoF.
F-test (1st 

NOTE: 8,315 observations removed because of NA and infinite values (LHS: 448, RHS: 8,037).



OLS estimation, Dep. Var.: log(pvi)
Observations: 38,070
Fixed-effects: year: 2,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|) 
log(deaths_i)                        -0.003686   0.008410 -0.438238  0.66176 
log(deaths_j)                        -0.001383   0.010638 -0.129996  0.89672 
C8EV_International_travel_controls_i -0.029246   0.021260 -1.375623  0.17071 
C8EV_International_travel_controls_j -0.016205   0.018940 -0.855575  0.39341 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.20355     Adj. R2: 0.064756
                Within R2: 1.02e-4 

In [20]:
fe_iv_no2021 = feols(log(pvi) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j|
 log(deaths_i) + log(deaths_j) ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df %>% filter(year != "2021")
 )

fe_iv_no2021 %>% summary(cluster = ~ i+j)

feols(log(pvi) ~  1 + log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j,
 data = df %>% filter(year != "2021")) %>% summary(cluster = ~ i+j)

NOTE: 10,033 observations removed because of NA and infinite values (LHS: 433, RHS: 6,512, IV: 4,510/7,371).



TSLS estimation - Dep. Var.: log(pvi)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(pvi)
Observations: 36,716
Fixed-effects: year: 2,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|) 
fit_log(deaths_i)                    -0.021265   0.024623 -0.863618  0.38899 
fit_log(deaths_j)                    -0.000103   0.018908 -0.005469  0.99564 
C8EV_International_travel_controls_i -0.042041   0.029512 -1.424564  0.15607 
C8EV_International_travel_controls_j -0.046876   0.028680 -1.634458  0.10397 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.13481     Adj. R2: 0.069122
                Within R2: 2.02e-4 
F-test (1st stage), log(deaths_i): stat = 1,339.5     , p < 2.2e-16 , on 6 and 36,359 DoF.
F-test (

NOTE: 9,697 observations removed because of NA and infinite values (LHS: 433, RHS: 9,442).



OLS estimation, Dep. Var.: log(pvi)
Observations: 37,052
Fixed-effects: year: 2,  i: 175,  j: 175
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error  t value Pr(>|t|)    
log(deaths_i)                        -0.008575   0.008337 -1.02860  0.30509    
log(deaths_j)                        -0.008519   0.008221 -1.03623  0.30153    
C8EV_International_travel_controls_i -0.035910   0.025286 -1.42016  0.15735    
C8EV_International_travel_controls_j -0.055573   0.027289 -2.03646  0.04322 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.13512     Adj. R2: 0.06926 
                Within R2: 3.371e-4

In [21]:
fe_iv_no2022 = feols(log(pvi) ~  1 +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j|
 log(deaths_i) + log(deaths_j) ~ 
# H6M_Facial_Coverings_i+H6M_Facial_Coverings_j
# +
C1M_School_closing_i+C1M_School_closing_j
+
median_age_i + median_age_j
+
population_density_i + population_density_j
# +
# average_surface_temperature_i + average_surface_temperature_j # Добавляет пропусков

 ,
 data = df %>% filter(year != "2022")
 )

fe_iv_no2022 %>% summary(cluster = ~ i+j)

feols(log(pvi) ~  1 + log(deaths_i) + log(deaths_j) +
 C8EV_International_travel_controls_i + C8EV_International_travel_controls_j | year+i+j,
 data = df %>% filter(year != "2022")) %>% summary(cluster = ~ i+j)

NOTE: 9,898 observations removed because of NA and infinite values (LHS: 409, RHS: 6,709, IV: 4,938/6,709).



TSLS estimation - Dep. Var.: log(pvi)
                  Endo.    : log(deaths_i), log(deaths_j)
                  Instr.   : C1M_School_closing_i, C1M_School_closing_j, median_age_i, median_age_j, population_density_i, population_density_j
Second stage: Dep. Var.: log(pvi)
Observations: 37,582
Fixed-effects: year: 2,  i: 171,  j: 171
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|) 
fit_log(deaths_i)                    -0.044444   0.033994 -1.307419  0.19284 
fit_log(deaths_j)                     0.012552   0.045345  0.276806  0.78227 
C8EV_International_travel_controls_i -0.003148   0.024339 -0.129330  0.89725 
C8EV_International_travel_controls_j -0.020780   0.026277 -0.790810  0.43016 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.09978     Adj. R2:  0.064375
                Within R2: -9.236e-4
F-test (1st stage), log(deaths_i): stat = 407.5    , p < 2.2e-16 , on 6 and 37,233 DoF.
F-test (1

NOTE: 9,898 observations removed because of NA and infinite values (LHS: 409, RHS: 9,657).



OLS estimation, Dep. Var.: log(pvi)
Observations: 37,582
Fixed-effects: year: 2,  i: 171,  j: 171
Standard-errors: Clustered (i & j) 
                                      Estimate Std. Error   t value Pr(>|t|) 
log(deaths_i)                         0.001408   0.007287  0.193175  0.84705 
log(deaths_j)                        -0.013597   0.009654 -1.408468  0.16082 
C8EV_International_travel_controls_i  0.005658   0.022578  0.250592  0.80243 
C8EV_International_travel_controls_j -0.027211   0.025703 -1.058682  0.29125 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 1.0992     Adj. R2: 0.065361
               Within R2: 1.316e-4

### Экспорт результатов в латех

In [22]:
etable(fe_no_iv, heterogenous_fe, fe_iv, vcov = ~i+j, signif.code=signif.code, fitstat=~ . + sargan.p + wh.p + ivf, tex=TRUE)

\begingroup
\centering
\begin{tabular}{lccc}
   \tabularnewline \midrule \midrule
   Dependent Variable: & \multicolumn{3}{c}{log(pvi)}\\
   Model:                                                          & (1)           & (2)           & (3)\\  
   \midrule
   \emph{Variables}\\
   log(deaths\_i)                                                  & -0.0034       &               & -0.0164\\   
                                                                   & (0.0060)      &               & (0.0252)\\   
   log(deaths\_j)                                                  & -0.0082       &               & -0.0094\\   
                                                                   & (0.0065)      &               & (0.0218)\\   
   C8EV\_International\_travel\_controls\_i                        & -0.0193       &               & -0.0246\\   
                                                                   & (0.0153)      &               & (0.0168)\\   
   C8EV\_International\_travel\_

In [23]:
etable(fe_no_iv, heterogenous_fe, fe_iv, vcov = ~i+j, signif.code=signif.code, fitstat=~ . + sargan.p + wh.p + ivf, tex=FALSE)

,,fe_no_iv,heterogenous_fe,fe_iv
,<chr>,<chr>,<chr>,<chr>
1,Dependent Var.:,log(pvi),log(pvi),log(pvi)
2,,,,
3,log(deaths_i),-0.0034 (0.0060),,-0.0164 (0.0252)
4,log(deaths_j),-0.0082 (0.0065),,-0.0094 (0.0218)
5,C8EV_International_travel_controls_i,-0.0193 (0.0153),,-0.0246 (0.0168)
6,C8EV_International_travel_controls_j,-0.0310. (0.0162),,-0.0288 (0.0175)
7,year2021,,-0.0831 (0.1078),
8,year2022,,-0.0393 (0.1215),
9,year2020 x log(deaths_i),,-0.0025 (0.0070),
